In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [ ]:
#输入图片是28*28
n_inputs = 28#输入一行的长度 取决于图片一行28像素
max_time = 28#一共28行
lstm_size = 100#100个隐层单元
n_classes = 10#10个分类
batch_size = 50#每批次50样本
n_batch = mnist.train.num_examples // batch_size#计算一共多少批次 

#x是一个占位符，这里的none表示第一个维度可以任意，取决于放多少张图片
x = tf.placeholder(tf.float32,[None,784])
#正确的标签，输出10个分类
y = tf.placeholder(tf.float32,[None,10])

#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置值
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X,Weights,Biases):
    #inputs = [batch_size,max_time,n_inputs]转换成[50,28,28]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL类型
    lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_size)
    #final_state[0]是cell state
    #final_state[1]是hidden_state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    #模型预测类别结果
    results = tf.nn.softmax(tf.matmul(final_state[1],Weights)+Biases)
    return results

#计算RNN的返回结果
prediction = RNN(x,weights,biases)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#把结果存放在一个布尔型列表中，返回一维张量中最大概率值的位置
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率，计算数组中的所高元素的平均值，相当于得到了模型的预测准确率
#因为correct_prediction 是一个布尔值链表，tf.cast将张量数据类型转化，例[1,0,1,1],精确度0.75=3/4
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(3):
        #每批次50 ，用总量/批次大小=批次次数，就是循环次数，结束后完成一次训练样本的训练
        for batch in range(n_batch):
            #每次提取batch_size个数据进行训练，共训练n_batch次
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ",Testing Accuracy=" + str(acc))